In [ ]:
!nvcc -V

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [ ]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "CUDA sur Google Colab fonctionne\n";
    return 0;
}

In [ ]:
%%cu
#include <cstdio>
#include <iostream>
 
using namespace std;
 
__global__ void maxi(int* a, int* b, int n)
{
    int block = 256 * blockIdx.x;
    int max = 0;
 
    for (int i = block; i < min(256 + block, n); i++) {
 
        if (max < a[i]) {
            max = a[i];
        }
    }
    b[blockIdx.x] = max;
}
 
int main()
{
 
    int n;
    n = 3 >> 2;
    int a[n];
 
    for (int i = 0; i < n; i++) {
        a[i] = rand() % n;
        cout << a[i] << "\t";
    }
 
    cudaEvent_t start, end;
    int *ad, *bd;
    int size = n * sizeof(int);
    cudaMalloc(&ad, size);
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
    int grids = ceil(n * 1.0f / 256.0f);
    cudaMalloc(&bd, grids * sizeof(int));
 
    dim3 grid(grids, 1);
    dim3 block(1, 1);
 
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
 
    while (n > 1) {
        maxi<<<grids, block>>>(ad, bd, n);
        n = ceil(n * 1.0f / 256.0f);
        cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
    }
 
    cudaEventRecord(end);
    cudaEventSynchronize(end);
 
    float time = 0;
    cudaEventElapsedTime(&time, start, end);
 
    int ans[2];
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);
 
    cout << "The maximum element is : " << ans[0] << endl;
 
    cout << "The time required : ";
    cout << time << endl;
}

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <math.h>

#define N 8192
#define THREADS_PER_BLOCK 256
#define SQRT_THREADS_PER_BLOCK sqrt(THREADS_PER_BLOCK)

struct event_pair
{
  cudaEvent_t start;
  cudaEvent_t end;
};

inline void start_timer(event_pair * p)
{
  cudaEventCreate(&p->start);
  cudaEventCreate(&p->end);
  cudaEventRecord(p->start, 0);
}


inline void stop_timer(event_pair * p, char * kernel_name)
{
  cudaEventRecord(p->end, 0);
  cudaEventSynchronize(p->end);
  
  float elapsed_time;
  cudaEventElapsedTime(&elapsed_time, p->start, p->end);
  printf("%s took %.4f ms\n",kernel_name, elapsed_time);
  cudaEventDestroy(p->start);
  cudaEventDestroy(p->end);
}

void checkCUDAError(const char*);
void random_floats(float *a, int n);
void print_array(float *a, int n, char *name);
int validate(float *a, float *ref, int n);

__global__ void simple_convolution1D_kernel(float* c, float* a, float* filter, int f, int n) {
  // A completer
}

__global__ void shared_convolution1D_kernel(float* c, float* a, float* filter, int f, int n) {
  // A completer
}

int main(void) {
	srand( time( NULL ) );

	float *a, *filter, *c;
	float *d_a, *d_filter, *d_c;
	int errors;
	
	//int f =
	//int n_c = 
	unsigned int filter_size = f * sizeof(float);
	unsigned int size = N * sizeof(float);
	unsigned int c_size = n_c * sizeof(float);

	event_pair timer;
	// Alloc space for device copies
	// A completer
	//cudaMalloc
	//cudaMalloc
	//cudaMalloc
	checkCUDAError("CUDA malloc");

	// Alloc space for host copies of a, b, c and setup input values
	// A completer
	//a = (float*)malloc(size);
	//filter = (float*)malloc(filter_size);
	//c = (float*)malloc(c_size);

	random_floats(a, N);
	random_floats(filter, f);
	print_array(a, N, "a");
	print_array(filter, f, "filter");

	// Copy inputs to device
	// A completer
	//cudaMemcpy
	//cudaMemcpy
	checkCUDAError("CUDA memcpy Host to Device");

	// Launch add() kernel on GPU
	dim3 blocksPerGrid((N + THREADS_PER_BLOCK -1)/THREADS_PER_BLOCK);
	dim3 threadsPerBlock(THREADS_PER_BLOCK);
	start_timer(&timer);
	simple_convolution1D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_c, d_a, d_filter, f, N);
	checkCUDAError("CUDA kernel");
	stop_timer(&timer,"Convolution 1D sur GPU");
	
	// Copy result back to host
	// A completer
	// cudaMemcpy
	checkCUDAError("CUDA memcpy Device to Host");

	print_array(c, n_c, "c");
	
	//start_timer(&timer);
	//shared_convolution1D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_cs, d_a, d_filter, f, N);
	//checkCUDAError("CUDA kernel");
	//stop_timer(&timer,"Convolution 1D shared sur GPU");


	// validate
	//errors = validate(c, cs, n_c);
	//printf("CUDA GPU result has %d errors.\n", errors);

	// Cleanup
	free(a); free(filter); free(c);
	cudaFree(d_a); cudaFree(d_filter); cudaFree(d_c);
	checkCUDAError("CUDA cleanup");

	return 0;
}

void checkCUDAError(const char *msg)
{
	cudaError_t err = cudaGetLastError();
	if (cudaSuccess != err)
	{
		fprintf(stderr, "CUDA ERROR: %s: %s.\n", msg, cudaGetErrorString(err));
		exit(EXIT_FAILURE);
	}
}

void random_floats(float *a, int n)
{
	for (unsigned int i = 0; i < n; i++){
			a[i] = (float)(rand() % 101);
	}
}

void print_array(float *a, int n, char*name){

	printf("%s : [ ",name);
	for (unsigned int i = 0; i < n; i++){
			printf("%.4f ",a[i]);
	}
	printf("]\n");
}

int validate(float *a, float *ref, int n){
	int errors = 0;
	for (unsigned int i = 0; i < n; i++){
		if (a[i] != ref[i]){
			errors++;
			fprintf(stderr, "ERROR at index %d: GPU result %f does not match CPU value of %f\n", i, a[i], ref[i]);
		}
	}

	return errors;
}

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>
#include <math.h>

#define N 2048
#define THREADS_PER_BLOCK_X 16
#define THREADS_PER_BLOCK_Y 16

struct event_pair
{
  cudaEvent_t start;
  cudaEvent_t end;
};

inline void start_timer(event_pair * p)
{
  cudaEventCreate(&p->start);
  cudaEventCreate(&p->end);
  cudaEventRecord(p->start, 0);
}


inline void stop_timer(event_pair * p, char * kernel_name)
{
  cudaEventRecord(p->end, 0);
  cudaEventSynchronize(p->end);
  
  float elapsed_time;
  cudaEventElapsedTime(&elapsed_time, p->start, p->end);
  printf("%s took %.4f ms\n",kernel_name, elapsed_time);
  cudaEventDestroy(p->start);
  cudaEventDestroy(p->end);
}

void checkCUDAError(const char*);
void random_floats(float *a, int n);
void print_array(float *a, int n, char *name);
int validate(float *a, float *ref, int n);

__global__ void simple_convolution2D_kernel(float* c, float* a, float* filter, int f, int n) {
  // A completer
}

__global__ void shared_convolution2D_kernel(float* c, float* a, float* filter, int f, int n) {
  // A completer
}

int main(void) {
	srand( time( NULL ) );

	float *a, *filter, *c;
	float *d_a, *d_filter, *d_c;
	int errors;
	
	//int f =
	//int n_c = 
	unsigned int filter_size = f * sizeof(float);
	unsigned int size = N * sizeof(float);
	unsigned int c_size = n_c * sizeof(float);

	event_pair timer;
	// Alloc space for device copies
	// A completer
	//cudaMalloc
	//cudaMalloc
	//cudaMalloc
	checkCUDAError("CUDA malloc");

	// Alloc space for host copies
	// A completer
	//a = (float*)malloc(size);
	//filter = (float*)malloc(filter_size);
	//c = (float*)malloc(c_size);

	random_floats(a, N);
	random_floats(filter, f);
	print_array(a, N, "a");
	print_array(filter, f, "filter");

	// Copy inputs to device
	// A completer
	//cudaMemcpy
	//cudaMemcpy
	checkCUDAError("CUDA memcpy Host to Device");

	// Launch kernel on GPU
	dim3 blocksPerGrid((N + THREADS_PER_BLOCK -1)/THREADS_PER_BLOCK);
	dim3 threadsPerBlock(THREADS_PER_BLOCK);
	start_timer(&timer);
	simple_convolution1D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_c, d_a, d_filter, f, N);
	checkCUDAError("CUDA kernel");
	stop_timer(&timer,"Convolution 1D sur GPU");
	
	// Copy result back to host
	// A completer
	// cudaMemcpy
	checkCUDAError("CUDA memcpy Device to Host");

	print_array(c, n_c, "c");
	
	//start_timer(&timer);
	//shared_convolution2D_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_cs, d_a, d_filter, f, N);
	//checkCUDAError("CUDA kernel");
	//stop_timer(&timer,"Convolution 1D shared sur GPU");


	// validate
	//errors = validate(c, cs, n_c);
	//printf("CUDA GPU result has %d errors.\n", errors);

	// Cleanup
	free(a); free(filter); free(c);
	cudaFree(d_a); cudaFree(d_filter); cudaFree(d_c);
	checkCUDAError("CUDA cleanup");

	return 0;
}

void checkCUDAError(const char *msg)
{
	cudaError_t err = cudaGetLastError();
	if (cudaSuccess != err)
	{
		fprintf(stderr, "CUDA ERROR: %s: %s.\n", msg, cudaGetErrorString(err));
		exit(EXIT_FAILURE);
	}
}

void random_floats(float *a, int n)
{
	for (unsigned int i = 0; i < n; i++){
			a[i] = (float)(rand() % 101);
	}
}

void print_array(float *a, int n, char*name){

	printf("%s : [ ",name);
	for (unsigned int i = 0; i < n; i++){
			printf("%.4f ",a[i]);
	}
	printf("]\n");
}

int validate(float *a, float *ref, int n){
	int errors = 0;
	for (unsigned int i = 0; i < n; i++){
		if (a[i] != ref[i]){
			errors++;
			fprintf(stderr, "ERROR at index %d: GPU result %f does not match CPU value of %f\n", i, a[i], ref[i]);
		}
	}

	return errors;
}